# Introduction to Pandas

*Pandas* is a high-performance Python library that allows us to easily use and manipulate data structures such as tables and series.

https://pandas.pydata.org/pandas-docs/stable/index.html

It connects directly to *NumPy* and *MatPlotLib* to manage numerical operations and plotting. This notebook is a quick introduction to the module. You are encouraged to explore more, for example starting here:

https://pandas.pydata.org/pandas-docs/stable/getting_started/intro_tutorials/index.html

https://pandas.pydata.org/pandas-docs/stable/user_guide/index.html

## Defining data frames (tables) and series in pandas

In [ ]:
import pandas as pd

We will define a table of information about the students of our class. We will try to reproduce the following tabular structure in *pandas*

| NIU     | Name              | Exam | Practicals |
|:-------:|:-----------------:|:----:|:----------:|  
| 111222  | Ned Stark         | 6.0  |    5.4     |   
| 123123  | Robert Baratheon  | 8.5  |    7.5     |   
| 234234  | Jaime Lannister   | 3.7  |    6.4     |   
| 345345  | Catelyn Stark     | 9.0  |    8.6     |   
| 456456  | Viserys Targaryen | 4.0  |    4.6     |   
| 567567  | Jon Snow          | 6.8  |    4.3     |   
| 678678  | Talisa Maegyr     | 7.6  |    0.0     |   

This table in pandas will be represented by a **data frame** which is implemented by the class `pandas.DataFrame`. Each column in a data frame is a **series** which is implemented by the `pandas.Series` object.

We will start with the first column, the NIU:

In [ ]:
seriesNIU = pd.Series( [111222, 123123, 234234, 345345, 456456, 567567, 678678] )
print (seriesNIU)

See how pandas added an extra column with indices. The structure of a series is like a dictionary, where each value has an index and can be retrieved by using its index. For example:

In [ ]:
seriesNIU[3]

Let's define the rest of the columns of our table in the same way

In [ ]:
seriesName = pd.Series( ["Ned Stark", "Robert Baratheon", "Jaime Lannister", "Catelyn Stark", "Viserys Targaryen", "Jon Snow", "Talisa Maegyr"] )
seriesExam = pd.Series( [6.0, 8.5, 3.7, 9.0, 4.0, 6.8, 7.6])
seriesPracticals = pd.Series( [5.4, 7.5, 6.4, 8.6, 4.6, 4.3, 0.0])

Now we can combine the series (the columns) together into a data frame (a table). The idea is the same, a dataframe can be thought of as a dictionary of columns, where each column has its title and its data.

In [ ]:
d = {"NIU": seriesNIU,
     "Name": seriesName,
     "Exam": seriesExam,
     "Practicals": seriesPracticals}

dfStudents = pd.DataFrame(d)
dfStudents

We can actually ask pandas to use our own custom indices. For example in this case, we could decide to use the NIUs as the indices. Let's tell pandas to use this column.

In [ ]:
dfStudents = dfStudents.set_index('NIU')
dfStudents

# An alternative way would be to make the change in place by calling:
# dfStudents.set_index('NIU', inplace = True)

We can ask for some quick information about a data frame by using `DataFrame.info()`. This tells us what columns we have, how many values there are in each column, their types, etc.

In [ ]:
dfStudents.info()

Since a data frame is like a dictionary of columns, we can refer to each column by its name. For example, the following will return the columns of exam scores as a series:

In [ ]:
scores = dfStudents["Exam"]
scores

See how our exam scores are now indexed using the NIU, as they should.

We can also filter specific rows of our table by using a mask. By mask I mean a list (or series) of `True` and `False` values. An easy way to create a mask is by using bollean operations, for example

In [ ]:
passScoresMask = scores > 5.0
passScoresMask

we can then use this mask to retrieve specific rows (the `True` ones):

In [ ]:
dfStudents[passScoresMask]

or directly:

In [ ]:
dfStudents[ dfStudents["Exam"] > 5.0 ]

Any boolean expression would do, for example get the members of the Stark family:

In [ ]:
dfStudents[ dfStudents["Name"].str.contains("Stark")]

Or a list of students with specific exam scores:

In [ ]:
dfStudents[ dfStudents["Exam"].isin([6.8, 8.5]) ]

Or the list of students that have passed both the exam and the practicals:

In [ ]:
dfStudents[ (dfStudents["Exam"]>5) & (dfStudents["Practicals"]>5) ]

Read the documentation for the function we used above, here:

https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.Series.str.contains.html

https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.isin.html

To get a specific value in the table, you can first get the column and then the row in the column. To do that you should use the corresponding indices - the names of the columns and the index for the row:

In [ ]:
dfStudents["Exam"][111222]

Alternatively, you can use integers to index a specific location, using the function `DataFrame.iloc`. Careful! Now the row is the first index and the column is the second one.

In [ ]:
dfStudents.iloc[0, 0]

In [ ]:
dfStudents.iloc[1, 0]

Adding a column is easy, you just have to tell pandas what you need in it (in the same way as you would add a new entry in a Python dictionary). Let's create a new column called "Total Mark" that is the average of the exam and the practicals grades:

In [ ]:
dfStudents["Total Mark"] = (dfStudents["Exam"] + dfStudents["Practicals"]) / 2
dfStudents

To change a value, you can use `DataFrame.at[]` or `DataFrame.iat[]` passing the row and then the column of the item you want to set:

In [ ]:
dfStudents.at[111222, "Name"] = "Davos Seaworth" # The indices here are the row and column keys

dfStudents.iat[1, 0] = "Brienne of Tarth" # the indices here are the row and column numbers

dfStudents

You can use the same way to add an new element, if you reference an element that does not exist it will be added in the list

In [ ]:
dfStudents.at["Starfleet_1234", "Name"] = "Jean Luc Piccard"
dfStudents

See that since we used a row index that does not exist, it was created for us. And since we only provided a value for the name, the rest are marked as "Not A Number" `NaN`. We can select all the rows that are NOT *NaN* using the function `notna()`.
> *To select all the values that ARE NaN you can use the function `isna()` or `isnull()`.*

In [ ]:
viewWithoutNaNs = dfStudents[dfStudents["Exam"].notna()] # This is just a different view of the same data - the data itself has not been changed
viewWithoutNaNs

Let's fix that new row by providing values for all the columns:

In [ ]:
dfStudents.at["Starfleet_1234", :] = ["Jean Luc Picard", 10.0, 10.0, 10.0]

# Or equivalently
# dfStudents.at["Starfleet_1234"] = ["Jean Luc Picard", 10.0, 10.0, 10.0]

dfStudents

Another way to add rows is to use `DataFrame.append()`

https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.append.html

In [ ]:
dfStudents.append(pd.Series(["Seven of Nine", 10.0, 10.0, 10.0], index = ["Name", "Exam", "Practicals", "Total Mark"], name = "Starfleet_5678"))

Check though that `append()` returns a new object, it does not change the original. Check it out:

In [ ]:
dfStudents

If we wanted to save this change, we should do this:

In [ ]:
dfStudents = dfStudents.append(pd.Series(["Seven of Nine", 10.0, 10.0, 10.0], index = ["Name", "Exam", "Practicals", "Total Mark"], name = "Starfleet_5678"))
dfStudents

Ok, Jean Luc and Seven of Nine are a bit out of place here, let's drop these two rows, literally:

In [ ]:
dfStudents.drop("Starfleet_1234", inplace = True) # If we do not use inplace = True, then the function will return a view of the data
dfStudents.drop("Starfleet_5678", inplace = True)
dfStudents

We can now ask pandas for some quick statistics for the columns that contain numbers, by calling the function `DataFrame.describe()`

In [ ]:
dfStudents.describe()

Or for a specific series

In [ ]:
dfStudents["Total Mark"].describe()

# Grading students (work in groups)

This is the same problem as you did in Problems 1 last week. You are now asked to repeat it, but using pandas.

You have been given a list of students in a CSV (Comma Separated Values) file, along with the evaluation scores for a series of works they have submitted. The table of data looks like this:

| student_id                           | Score  | Submission_Time  |
| ------------------------------------ |:------:| ----------------:|
| B73F2C11-70F0-E37D-8B10-1D20AFED50B1 | 92.7   | 02/11/2015 06:55 |
| 98A0FAE0-A19A-13D2-4BB5-CFBFD94031D1 | 86.8   | 29/11/2015 14:57 |
| D0F62040-CEB0-904C-F563-2F8620916C4E | 85.5   | 09/01/2016 05:36 |
| FFDF2B2C-F514-EF7F-6538-A6A53518E9DC | 86.0   | 30/04/2016 06:50 |
| 5ECBEEB6-F1CE-80AE-3164-E45E99473FB4 | 64.8   | 13/12/2015 17:06 |
| ... | ... | ... |


In this exercise we will import and manipulate this data. A lot of code is already given. When code is asked for, this is indicated <font color=blue>in blue</font>.

<br>

First, let's get the data in from a csv file. To do this with pandas we will use the following code. At the time of opening the file, we can specify to panda how to treat different columns. Here we say that want the column called "student_id" to be the index columns, and also that the submission time should be parsed as a date

In [ ]:
grades = pd.read_csv("grades.csv", parse_dates=['Submission_Time'], dayfirst=True)

In [ ]:
grades.info()

We read 2315 rows and 3 columns. Note that the Submission_Time column is of type `datetime64`. This is a type of numpy, you can read more about it here

https://numpy.org/doc/stable/reference/arrays.datetime.html

Let's have a look at the first few entries

In [ ]:
grades.head()

<br>

<font color="blue">**Exercise** Print out the students that scored below 20 or above 99.5</font>

In [ ]:
# Your Code Here


<font color=blue>**Exercise** Filter out the students that submitted their work after 2015-12-31. We give you the deadline definition below.</font>

In [ ]:
import numpy as np
deadline = np.datetime64('2015-12-31')

In [ ]:
# Your Code Here


<br>

Now let's find the best performing and the worst performing student (of the ones that submitted their work on time).

<font color=blue>**Exercise** Find the minimum and the maximum Score.</font>

In [ ]:
# Your Code Here


<br>

<font color=blue>**Exercise** Retrieve the students with the lowest and highest scores</font>

In [ ]:
# Your Code Here


<br>

<font color=blue>**Exercise** Find the average the median and the quantile values for the class scores.</font>

> *Hint: use `describe()`*

In [ ]:
# Your Code Here


<br>

<font color=blue>**Exercise** Plot the histogram of scores of all students.</font>

In [ ]:
# Your Code Here
